# 必要なモジュールのインストール

In [1]:
# %%bash
# pip install spacy ginza

In [2]:
# %%bash
# pip install sudachipy sudachidict_core

In [3]:
from sudachipy import tokenizer
from sudachipy import dictionary
tokenizer_obj = dictionary.Dictionary().create()

In [4]:
for token in tokenizer_obj.tokenize("トンネルを抜けると雪国だった。"):
    print(token.surface(), token.part_of_speech())

トンネル ['名詞', '普通名詞', 'サ変可能', '*', '*', '*']
を ['助詞', '格助詞', '*', '*', '*', '*']
抜ける ['動詞', '非自立可能', '*', '*', '下一段-カ行', '終止形-一般']
と ['助詞', '接続助詞', '*', '*', '*', '*']
雪国 ['名詞', '普通名詞', '一般', '*', '*', '*']
だっ ['助動詞', '*', '*', '*', '助動詞-ダ', '連用形-促音便']
た ['助動詞', '*', '*', '*', '助動詞-タ', '終止形-一般']
。 ['補助記号', '句点', '*', '*', '*', '*']


# データのダウンロード

In [5]:
%%bash
# WRIME: 主観と客観の感情分析データセット https://github.com/ids-cv/wrime
curl -o ../data/wrime.tsv https://raw.githubusercontent.com/ids-cv/wrime/master/wrime.tsv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9264k  100 9264k    0     0  3792k      0  0:00:02  0:00:02 --:--:-- 3790k


In [6]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [63]:
import pandas as pd
import spacy
nlp = spacy.load('ja_ginza')

In [67]:
doc = nlp("トンネルを抜けるとそこは雪国でした")

In [76]:
for token in doc:
    print(token, "\t", token.vector[:3])

トンネル 	 [-0.14804362 -0.01284575  0.18524638]
を 	 [-0.19509408 -0.13202968 -0.01801249]
抜ける 	 [0. 0. 0.]
と 	 [-0.07328872 -0.11246356 -0.06079749]
そこ 	 [0. 0. 0.]
は 	 [-0.05035316 -0.15731327 -0.08336552]
雪国 	 [ 0.14457944 -0.04334065  0.07879572]
でし 	 [ 0.09235096 -0.15087031  0.04086336]
た 	 [-0.03423065 -0.15928707  0.00305681]


In [81]:
doc.vector[:3]

array([-0.0293422 , -0.08535002,  0.01619853], dtype=float32)

In [64]:
df = pd.read_csv("../data/wrime.tsv", sep='\t')

In [65]:
# (1) `Avg. Readers_Joy` が0より上、あるいは `Avg. Readers_Sadness` が0より上なデータだけに限定
df = df[(df['Avg. Readers_Joy'] > 0) | (df['Avg. Readers_Sadness'] > 0)]
# (2) 'Avg. Readers_Joy' - `Avg. Readers_Sadness` を行い、1つの値にする
# 例) 喜びの平均3、悲しみの平均1のときには 3 - 1 = 2となる。
df['JoySadness'] = df['Avg. Readers_Joy'] - df['Avg. Readers_Sadness']
# (3) その値が0より上であれば"ポジティブ"、0以下であれば"ネガティブ"になるようにする
df['PosiNega'] = np.where(df['JoySadness'] > 0, "ポジティブ", "ネガティブ")
# (4) `Sentence` と `PosiNega` だけを用いるのでカラムを限定
df = df[['Sentence', 'PosiNega']]
# (5) 先頭の10000行に限定
df = df.head(10000)

In [66]:
df.head(5)

,Sentence,PosiNega
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,ネガティブ
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,ポジティブ
3,眠い、眠れない。,ネガティブ
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,ポジティブ
5,表情筋が衰えてきてる｡まずいな…,ネガティブ


In [13]:
vectors = []
for i, sentence in df['Sentence'].iteritems():
    doc = nlp(sentence)
    vectors.append(doc.vector)

In [47]:
X = vectors
y = df['PosiNega'].map({0: "ネガティブ", 1: "ポジティブ"})

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [49]:
clf = RandomForestClassifier()

In [50]:
clf.fit(X_train, y_train);

In [51]:
clf.score(X_train, y_train)

0.998

In [52]:
clf.score(X_test, y_test)

0.74025

In [53]:
from joblib import dump, load

In [54]:
dump(clf, '../models/classify_review.joblib') 

['../models/classify_review.joblib']

In [55]:
estimator = load('../models/classify_review.joblib')

In [59]:
estimator.predict([doc.vector])

array(['ネガティブ'], dtype=object)